In [1]:
from utils.nn_utils import get_layer_by_name, set_precision
from utils.nn_utils import quantize_conv2d, q_freeze_conv2d, q_unfreeze_conv2d
from utils.exp_utils import print_config, setup_exp, iterate_all_configs, elaborate_configs
from utils.exp_utils import *
from utils.nn_data import act_dim_dict, load_act_dim_data
import utils.exp_utils as exp_utils

from utils.utils import set_timezone, setup_log_dir, create_logger, save_dict, load_dict
from utils.utils import setup_dir, logger_info, logger_newline, logger_bold, logger_lvl1, logger_lvl2, green

import torch
import torch.nn as nn
import torch.optim

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from time import time
from multiprocessing import set_start_method

from utils.nn_utils import get_model
from dnn_model.models.quan_ops import Conv2d_Q



In [2]:

model = get_model('resnet20qe1', [4], [4], 10, [0], torch.device(f"cuda:0"))




In [11]:
layer_config_path = "./exp_results/resnet20qe1__mixed_SLC_4bit_array64/run_2_2024-09-19_15-31-55/layer_config.yaml"
layer_config = load_dict(layer_config_path)

layer_names = list(layer_config.keys())
print(layer_names)

numel_weight_list = []
prec_list = []

for name in layer_names:
    layer = get_layer_by_name(model, name)
    num_weights = layer.weight.numel()
    prec = layer_config[name]['precision']

    numel_weight_list.append(num_weights)
    prec_list.append(prec)

numel_weight_list = np.array(numel_weight_list)
numel_weight_list = numel_weight_list / numel_weight_list.sum()

prec_list = np.array(prec_list)
avg_prec = (numel_weight_list * prec_list).sum()

print(np.log2(avg_prec))


['layers.0.conv0', 'layers.0.conv1', 'layers.1.conv0', 'layers.1.conv1', 'layers.2.conv0', 'layers.2.conv1', 'layers.3.conv0', 'layers.3.conv1', 'layers.4.conv0', 'layers.4.conv1', 'layers.5.conv0', 'layers.5.conv1', 'layers.6.conv0', 'layers.6.conv1', 'layers.7.conv0', 'layers.7.conv1', 'layers.8.conv0', 'layers.8.conv1']
4.368431197661214


In [44]:
layer = get_layer_by_name(model, layer_name_list[0])
w_q_int = layer.w_q_int.detach().cpu()
hist, bin_edges = np.histogram(w_q_int.detach().cpu().numpy(), bins=np.array(range(-7, 7)) + 0.5)


In [46]:
hist, bin_edges

(array([284, 120, 162, 138, 136, 169, 328, 167, 130, 139, 191, 124, 216]),
 array([-6.5, -5.5, -4.5, -3.5, -2.5, -1.5, -0.5,  0.5,  1.5,  2.5,  3.5,
         4.5,  5.5,  6.5]))

In [47]:
total_hist = hist/np.sum(hist)
total_hist = np.cumsum(total_hist)
total_hist = np.insert(total_hist, 0, 0)
total_hist[-1] = 1

print(total_hist)

[0.         0.12326389 0.17534722 0.24565972 0.30555556 0.36458333
 0.43793403 0.58029514 0.65277778 0.70920139 0.76953125 0.85243056
 0.90625    1.        ]


In [52]:
random_array = np.random.rand(1_000_000)
random_array = np.digitize(random_array, total_hist)

In [53]:
random_array = random_array - 7
hist, bin_edges = np.histogram(random_array, bins=np.array(range(-7, 7)) + 0.5)

In [55]:
hist = hist/np.sum(hist)
hist

array([0.123268, 0.05222 , 0.070539, 0.06023 , 0.058725, 0.073354,
       0.142062, 0.072862, 0.056265, 0.060653, 0.082131, 0.053905,
       0.093786])

In [2]:
import os
import os.path as osp

base_path = './exp_result'
exp_name = 'exp1'
exp_path = osp.join(base_path, exp_name)
run_name = 'run1'
run_path = osp.join(exp_path, run_name)

if os.path.exists(exp_path):
    raise ValueError('Experiment already exists')

os.makedirs(exp_path)



In [1]:
from utils.exp_utils import save_dict, load_dict

base_nn_config = {
    'lead_gpu_idx': 0,
    'all_gpu_idx': [0],
    'acti_bit': [4],
    'weight_bit': [1, 2, 4, 8],
    'batch_size': 256,
    'num_workers': 16,
    'model': 'resnet50q',
    'half_prec': True,
    
}

save_dict(base_nn_config, './base_nn_config.yml')

In [2]:
R1C4 = {
    'name': 'R1C4', 'r_start': 0,
    'num_row_grouping': 1, 'num_col_grouping': 4}
R2C2 = {
    'name': 'R2C2', 'r_start': 1,
    'num_row_grouping': 2, 'num_col_grouping': 2}
R2C4 = {
    'name': 'R2C4', 'r_start': 1,
    'num_row_grouping': 2, 'num_col_grouping': 4}

R1C2 = {
    'name': 'R1C2', 'r_start': 0,
    'num_row_grouping': 1, 'num_col_grouping': 2}
R2C2 = {
    'name': 'R2C2', 'r_start': 1,
    'num_row_grouping': 2, 'num_col_grouping': 2}
R2C1 = {
    'name': 'R2C1', 'r_start': 1,
    'num_row_grouping': 2, 'num_col_grouping': 1}

base_nn_config = {
    'lead_gpu_idx': 0,
    'all_gpu_idx': [0],
    'acti_bit': [4],
    'weight_bit': [1, 2, 4, 8],
    'batch_size': 256,
    'num_workers': 16,
    'model': 'resnet50q',
    'half_prec': True,
    
}

base_imc_config = {
    'imc_array_dim': 512,
    'rc_config': [
        R1C4,
        R2C2,
        R2C4
        ],
    'rc_config_prec': [16, 7, 31],
    'cell_precision': 2,
    'shift_base': 2,
    'p_saf0': 0.0175,
    'p_saf1': 0.0904,
    'num_parallel_processes': 20
}

In [3]:
imc_iter_keys = {
    'imc_array_dim': [512, 256, 128],
    'cell_precision': [2,4],
}

imc_iter_comb = list(itertools.product(*imc_iter_keys.values()))

In [5]:
def iterate_all_configs(fixed_config, iter_config):
    iter_combinations = list(product(*iter_config.values()))

    all_configs = []
    
    for combo in iter_combinations:
        config = fixed_config.copy()
        for key, value in zip(iter_config.keys(), combo):
            config[key] = value
        all_configs.append(config)

    return all_configs

In [28]:
fixed_config = {
    'imc_array_dim': 512,
    'rc_config': [
        R1C4,
        R2C2,
        R2C4
        ],
    'cell_precision': 2,
    'p_fault': 0.10,
    'num_parallel_processes': 20
}

iter_configs = {
    'imc_array_dim': [512, 256, 128],
    'rc_config': [
        [R1C4],
        [R1C4, R2C2],
        [R1C4, R2C4],
        [R1C4, R2C2, R2C4],
    ]
}

In [32]:
from rc_grouping.rccodes import calc_q_lvl_RC
from utils.exp_utils import print_config

def elaborate_configs(all_configs):
    for config in all_configs:
        # fix shift_base = cell_precision
        config['shift_base'] = config['cell_precision']

        # fix saf0 and saf1 ratio
        if config['p_fault'] is None:
            config['p_saf0'] = 0.0175
            config['p_saf1'] = 0.0904
        else:
            p_ratio_total = 0.0175 + 0.0904
            config['p_saf0'] = 0.0175/p_ratio_total * config['p_fault']
            config['p_saf1'] = 0.0904/p_ratio_total * config['p_fault']

        # compute rc_config_prec based on rc_config
        rc_config_prec = []
        for rc in config['rc_config']:
            prec = calc_q_lvl_RC(
                R=rc['num_row_grouping'], 
                C=rc['num_col_grouping'],
                mem_q_lvl=config['cell_precision'],
                shift_base=config['shift_base'])
            rc_config_prec.append(prec)
        config['rc_config_prec'] = rc_config_prec

    return all_configs

from utils.utils import set_timezone, setup_log_dir, create_logger
from utils import exp_utils

set_timezone('Asia/Seoul')
log_dir = setup_log_dir('_logs')
logger = create_logger('_test', log_dir)
exp_utils.logger = logger

In [33]:
all_combo = iterate_all_configs(fixed_config, iter_configs)
all_combo = elaborate_configs(all_combo)

for config in all_combo:
    print_config(config)

2024-09-09 12:07:02 - _test - INFO -      imc_array_dim: 512
2024-09-09 12:07:02 - _test - INFO -      imc_array_dim: 512
2024-09-09 12:07:02 - _test - INFO -      imc_array_dim: 512
2024-09-09 12:07:02 - _test - INFO -      imc_array_dim: 512
2024-09-09 12:07:02 - _test - INFO -      imc_array_dim: 512
2024-09-09 12:07:02 - _test - INFO -      rc_config: [{'name': 'R1C4', 'r_start': 0, 'num_row_grouping': 1, 'num_col_grouping': 4}]
2024-09-09 12:07:02 - _test - INFO -      rc_config: [{'name': 'R1C4', 'r_start': 0, 'num_row_grouping': 1, 'num_col_grouping': 4}]
2024-09-09 12:07:02 - _test - INFO -      rc_config: [{'name': 'R1C4', 'r_start': 0, 'num_row_grouping': 1, 'num_col_grouping': 4}]
2024-09-09 12:07:02 - _test - INFO -      rc_config: [{'name': 'R1C4', 'r_start': 0, 'num_row_grouping': 1, 'num_col_grouping': 4}]
2024-09-09 12:07:02 - _test - INFO -      rc_config: [{'name': 'R1C4', 'r_start': 0, 'num_row_grouping': 1, 'num_col_grouping': 4}]
2024-09-09 12:07:02 - _test - INFO 